In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.utils import compute_class_weight
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [ ]:
X_attacker = np.load(Path("ClosestCarCSVsModel2/np_attacker.npy"))
X_normal = np.load(Path("ClosestCarCSVsModel2/np_normal.npy"))
# X_attacker = torch.load(Path("ClosestCarCSVsModel2/df_attacker.pt"))
# X_normal = torch.load(Path("ClosestCarCSVsModel2/df_normal.pt"))

X_normal_split, p = train_test_split(X_normal, test_size=0.70, random_state=42, shuffle=True)
print(X_normal_split[0])
print(X_attacker[0])

In [ ]:
y = np.asarray([0]*X_normal_split.shape[0] + [1]*X_attacker.shape[0]).astype('float32')
X = np.concatenate((X_normal_split, X_attacker), axis=0)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Define the training and validation data
# X = [
#     [[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3],
#      [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4],
#      [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5],
#      [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
#      [0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]],
#     [[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4],
#      [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5],
#      [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
#      [0.5, 0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
#      [0.6, 0.7, 0.8, 0.9, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]]
# ]

# y = [0, 1]

# X = torch.tensor(X, dtype=torch.long, device=device)
# y = torch.tensor(y, dtype=torch.long, device=device)

# Create the dataset and dataloader
dataset = TimeSeriesDataset(X, y)
dataloader = DataLoader(dataset, batch_size=2000, shuffle=True)

# Define the hyperparameters
input_dim = 12
hidden_dim = 64
num_layers = 2
num_classes = 2
lr = 1e-3
num_epochs = 10

# Initialize the model, optimizer, and loss function
model = TimeSeriesClassificationModel(input_dim, hidden_dim, num_layers, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(dataloader):
        sequences = sequences.to(device)
        labels = labels.to(device)
        labels = labels.to(torch.long)
#         print(labels)
        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Batch {i}')
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

In [ ]:
dataset = TimeSeriesDataset(X_test, y_test)
dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)

model.eval()
with torch.no_grad():
    for sequences, labels in dataloader:
        sequences = sequences.to(device)
        labels = labels.to(device)



        # Forward pass
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)

        # Calculate accuracy
        total = labels.size(0)
        correct = (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")




In [ ]:
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))